In [ ]:
from os.path import isfile
from dask.distributed import Client
import dask.bag as db
import pandas as pd
import numpy as np

In [ ]:
avroFiles = !ls dataAvro/*.avro
avroFiles

In [ ]:
fthrFiles = !ls data/*.fthr
fthrFiles

In [ ]:
newAvroFiles = [f for f in avroFiles if not isfile(f.replace('dataAvro', 'data').replace('avro', 'fthr'))]
print(len(avroFiles) - len(newAvroFiles) - len(fthrFiles))
len(newAvroFiles)

In [ ]:
def avro2fthr(avroDict):    
    fs = avroDict['samplerate']
    
    fileSaveName = 'data/' + avroDict['id'] + '_' + str(avroDict['timestamp']) + '.fthr'
    if len(avroDict['adc_signal_mv']) == 1:
        x = avroDict['adc_signal_mv'][0]
        pd.DataFrame({
            'time_ms': np.arange(len(x)) / fs * 1000,
            'adc_signal_mv': x
        }).to_feather(fileSaveName)
    else:
        x0 = avroDict['adc_signal_mv'][0]
        x1 = avroDict['adc_signal_mv'][1]
        x2 = avroDict['adc_signal_mv'][2]
        x3 = avroDict['adc_signal_mv'][3]
        pd.DataFrame({
            'time_ms': np.arange(len(x0)) / fs * 1000,
            'ch0_adc_signal_mv': x0,
            'ch1_adc_signal_mv': x1,
            'ch2_adc_signal_mv': x2,
            'ch3_adc_signal_mv': x3,
        }).to_feather(fileSaveName)

In [ ]:
with Client() as client:
    display(client)
    d = db.read_avro(newAvroFiles, blocksize=None)
    df = d.map(avro2fthr).compute()